<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# この章では、BERTを使って穴埋めタスクを行う

